In [22]:
import pandas as pd
import os
from dags.virgo_functions.prepro_data_forecast import split_data, WindowGenerator
from dags.virgo_functions.configs import data_configs

import mlflow.pyfunc
from mlflow import MlflowClient
from dateutil.relativedelta import relativedelta
#import tensorflow


split_config = data_configs.split_config

In [3]:
def call_params_from_mlflow(self, call_model = False):

    registered_model_name = f'{self.stock_code}_models'
    filter_registered_model_name = "name='{}'".format(registered_model_name)

    client = MlflowClient()
    latest_version_info = client.get_latest_versions(registered_model_name, stages=["Production"])
    latest_production_version = latest_version_info[0].version

    for mv in client.search_model_versions(filter_string = f'{filter_registered_model_name}'):
        mv = dict(mv)
        if mv['version'] == latest_production_version:
            run_id = mv['run_id']
            break

    model_params = client.get_run(run_id).data.params
    self.lag_days = int(model_params['lag_days'])
    self.input_length = int(model_params['input_length'])
    self.OUT_STEPS = int(model_params['OUT_STEPS'])

    model_version = latest_production_version
    self.model_version = model_version
    print(self.model_version, self.lag_days, self.input_length, run_id)

    if call_model:
        
        path = os.getcwd()
        model = mlflow.pyfunc.load_model(
            model_uri=f"{path}/mlruns/{run_id}/artifacts/{self.stock_code}-run",
            suppress_warnings = True
        )
        print('method 3 worded')

        self.model = model 

In [4]:

main_path = os.getcwd()

class call_mlflow():
    def __init__(self, stock_code):
        self.stock_code = stock_code
        call_mlflow.call_params_from_mlflow = call_params_from_mlflow


In [5]:
main_path = 'c:\\Users\\Miguel\\virgo_airflow'

In [6]:

target_data = pd.read_csv(f'{main_path}/tmp_data/target_data.csv')
raw_data = pd.read_csv(f'{main_path}/tmp_data/raw_data.csv')
stock_codes = target_data.stock_code.unique()


In [ ]:

dataframes = list()

for stock_code in stock_codes:
    stock_target_data = target_data[target_data.stock_code == stock_code]
    stock_target_data = stock_target_data.drop(columns = ['stock_code'])

    stock_raw_data = raw_data[raw_data.stock_code == stock_code]
    stock_raw_data = stock_raw_data.drop(columns = ['stock_code'])

    split_object = split_data(stock_target_data, split_config)
    mlflow_object = call_mlflow(stock_code)
    mlflow_object.call_params_from_mlflow(call_model=True)

    column_indices = split_object.column_indices
    n = split_object.ndata
    train_df = split_object.train_df
    val_df  = split_object.val_df
    test_df = split_object.test_df
    num_features = split_object.num_features
    ### scaling
    split_object.scaling()
    train_mean = split_object.train_mean
    train_std = split_object.train_std
    train_df = split_object.train_df
    val_df  = split_object.val_df
    test_df = split_object.test_df

    wide_window = WindowGenerator(
        total_data = stock_target_data, 
        raw_stock = stock_raw_data,
        train_df=train_df, 
        val_df=val_df, 
        test_df=test_df,
        input_width= mlflow_object.input_length, 
        label_width= mlflow_object.OUT_STEPS, 
        shift= mlflow_object.OUT_STEPS,
        label_columns=['stock_logdif']
    )

    predictions, data_ = wide_window.expected_return(
            plot_col='stock_logdif',
            model = mlflow_object.model,
            train_mean = train_mean, 
            train_std = train_std,
            )

    final_result, some_history, prep_predictions = wide_window.get_futur_prices(
        predictions= predictions,
        steps_futur = mlflow_object.OUT_STEPS, 
        stock_code= stock_code,
        OUT_STEPS= mlflow_object.OUT_STEPS,
        train_std= train_std,
        train_mean = train_mean,
        lag_days= mlflow_object.lag_days,
        )
    final_result['stock_code'] = stock_code

    dataframes.append(final_result)

dataset = pd.concat(dataframes)

dataset.to_csv(f'{main_path}/tmp_data/forecasts.csv', header = True, index = False)



In [19]:
stock_raw_data

,Date,stock_price,stock_stv,stock_Volume,smoothed_Close
0,2020-08-24,33.688175,0.274641,25011947,33.431750
1,2020-08-25,33.315205,0.224390,26221201,33.508677
2,2020-08-26,33.002960,0.273399,24083689,33.450635
3,2020-08-27,32.838169,0.345726,22156556,33.316333
4,2020-08-28,32.881527,0.384367,32830519,33.172884
...,...,...,...,...,...
651,2023-03-27,40.220001,0.297850,20262800,40.304374
652,2023-03-28,39.990002,0.314945,13537500,40.279062
653,2023-03-29,40.250000,0.230341,14429500,40.192344
654,2023-03-30,40.380001,0.159135,12632800,40.209641


In [29]:
pd.to_datetime(stock_raw_data['Date']).max() + relativedelta(days = 5+1)

Timestamp('2023-04-06 00:00:00')

In [24]:
[ stock_raw_data['Date'].max() + relativedelta(days = i+1) for i in  range(7)],

TypeError: can only concatenate str (not "relativedelta") to str

In [17]:
wide_window.total_data

,smoothed_Close,roll_std,roll_log_Volume,roll_pos_noise_count,roll_neg_noise_count,dist_max,dist_min,stock_logdif,roll_up_yield,roll_low_yield,roll_stock_logdif,lag_30_stock_logdif,lag_30_roll_stock_logdif,lag_30_roll_up_yield,lag_30_roll_low_yield,stock_code
0,31.705238,0.228634,16.883161,2.0,8.0,0.0,11.0,0.016996,3.0,0.0,0.007756,-0.025555,-0.025555,0.0,1.0,PFE
1,31.766694,0.227121,16.883303,2.0,8.0,0.0,12.0,0.009551,3.0,0.0,0.010117,-0.013103,-0.019329,0.0,2.0,PFE
2,31.817518,0.232882,16.851682,1.0,9.0,0.0,13.0,0.000271,3.0,0.0,0.008939,-0.006857,-0.015172,0.0,3.0,PFE
3,31.873911,0.230713,16.851323,1.0,9.0,0.0,13.0,0.001901,3.0,0.0,0.003908,-0.026226,-0.015395,0.0,3.0,PFE
4,31.902977,0.218918,16.830469,2.0,8.0,0.0,13.0,-0.007360,2.0,1.0,-0.001729,-0.018906,-0.017330,0.0,3.0,PFE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241,70.315516,1.406944,14.810370,8.0,2.0,13.0,0.0,0.008494,1.0,2.0,-0.007001,-0.007326,-0.012138,0.0,3.0,PCAR
1242,70.502862,1.284728,14.700369,7.0,3.0,13.0,1.0,0.028664,2.0,1.0,0.005122,0.017919,0.000728,1.0,2.0,PCAR
1243,70.904002,1.173937,14.682372,6.0,4.0,13.0,2.0,0.034450,3.0,0.0,0.023869,0.006030,0.005541,2.0,1.0,PCAR
1244,71.337801,1.089023,14.577569,5.0,5.0,13.0,3.0,0.017744,3.0,0.0,0.026953,0.029396,0.017782,3.0,0.0,PCAR


15